In [37]:
import secret_data
import json
import tweepy
import pandas as pd

In [38]:
#Start getting keys & secrets for running Twitter user, you will need your own user with details saved in a file named 'secret_data.txt' to run this
CONSUMER_KEY = secret_data.CONSUMER_KEY
CONSUMER_SECRET = secret_data.CONSUMER_SECRET
ACCESS_TOKEN = secret_data.ACCESS_TOKEN
ACCESS_SECRET = secret_data.ACCESS_SECRET
#End getting keys & secrets for running Twitter user

In [39]:
#Start cache setup
CACHE_FNAME = 'twitter_cache.json'
try:
    cache_file = open(CACHE_FNAME, 'r')
    cache_contents = cache_file.read() #this is a str
    CACHE_DICTION = json.loads(cache_contents) #this is a dict
    cache_file.close()
except:
    CACHE_DICTION = {}
#End cache setup

In [40]:
#Start OAuth code
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)
#End OAuth code

In [41]:
#file containing tweet ids
# FILENAME = 'test_tweet_ids_10.txt'
FILENAME = 'test_tweet_ids.txt'

In [42]:
#Start funct to grab vars from Tweet
def get_tweet_vars(tweet):
    try:
        #set a number of vars for potential use
        tweet_text = tweet['text']
        tweet_in_reply_to_status_id_str = tweet['in_reply_to_status_id_str']
        tweet_in_reply_to_screen_name = tweet['in_reply_to_screen_name']
        tweet_entities_hashtags = tweet['entities']['hashtags']
        tweet_entities_symbols = tweet['entities']['symbols']
        tweet_entities_user_mentions = tweet['entities']['user_mentions']
        tweet_entities_urls = tweet['entities']['urls']

    except Exception as e:
        print('Exception in get_tweet_vars:{}\nProblematic Tweet:{}\n\n'.format(e, tweet))
#End funct to grab vars from Tweet

In [43]:
#Start funct for cache check
def get_tweet(found_id):
    if found_id in CACHE_DICTION:
        #if we get strange results in cache like missing child tweets then we may need to add the call to get_tweet() here
        # return CACHE_DICTION[found_id]
        if CACHE_DICTION[found_id]: #simple check if dict is populated or not
            get_tweet_vars(CACHE_DICTION[found_id])
    else:
        try:
            resp = api.get_status(found_id) #resp is a class 'tweepy.models.Status'
            json_str = json.dumps(resp._json) #json_str var is str type
            json_obj = json.loads(json_str) #json_obj var is dict type
        except Exception as e:
            json_obj = {}
            message = 'No worries. Empty entry has been made in cache.'
            print('Exception in get_tweet:{}\nProblematic Tweet:{}\n\n{}\n'.format(e, found_id,message))
        CACHE_DICTION[found_id] = json_obj #creating new entry in cache dict where key = 'found_id' & value = 'json_obj' which is a dict
        dumped_json_cache = json.dumps(CACHE_DICTION)
        fw = open(CACHE_FNAME,"w")
        fw.write(dumped_json_cache)
        fw.close() # close the open file

        #start recursive call to get_tweet() a.k.a. this funct if current tweet is in reply to another tweet
        if CACHE_DICTION[found_id]['in_reply_to_status_id_str'] is not None:
            in_reply_id = CACHE_DICTION[found_id]['in_reply_to_status_id_str']
            get_tweet(in_reply_id) 
        #end recursive call to get_tweet()

        # return CACHE_DICTION[found_id]
        if CACHE_DICTION[found_id]: #simple check if dict is populated or not
            get_tweet_vars(CACHE_DICTION[found_id])
#End funct for cache check

In [49]:
#Start funct to read in Tweet IDs from file
def read_in_tweet_ids():
    with open(FILENAME, 'r') as infile:
        for line in infile:
            found_id = line.split('\t')[0]
            label = line.split('\t')[2] #label is needed for training of model
            try:
                get_tweet(found_id)
            except:
                pass #having exception message print here was causing duplicate messages when get_tweet failed
#End funct to read in Tweet IDs from file

In [50]:
check = read_in_tweet_ids()

0

0

0

0

0

1

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

1

1

0

0

1

1

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

1

0

1

0

0

0

1

1

1

0

0

1

0

0

0

0

1

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0



IndexError: list index out of range

In [45]:
#convert CACHE_DICTION to DF & transpose so it's more intuitive
cache_df = pd.DataFrame(CACHE_DICTION).transpose()

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,place,possibly_sensitive,possibly_sensitive_appealable,retweet_count,retweeted,retweeted_status,source,text,truncated,user
343909778008973312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352823276889837570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339867818843594756,None,None,Wed May 29 22:16:17 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,339867818843594756,339867818843594756,DoctorChristian,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@DoctorChristian scared to start fluoxetine, w...",False,"{'id': 246979971, 'id_str': '246979971', 'name..."
349294537367236611,None,None,Mon Jun 24 22:34:41 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349294537367236611,349294537367236611,No2Speciesism1,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@IntuitiveGal1 ok, if you stopped taking the L...",False,"{'id': 149749939, 'id_str': '149749939', 'name..."
349294117076996096,None,None,Mon Jun 24 22:33:01 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349294117076996096,349294117076996096,K_Mowers,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@K_Mowers I think very tired; maybe more depre...,False,"{'id': 438490210, 'id_str': '438490210', 'name..."
349293570693402625,None,None,Mon Jun 24 22:30:51 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349293570693402625,349293570693402625,No2Speciesism1,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@IntuitiveGal1 well, how exactly is it making ...",False,"{'id': 149749939, 'id_str': '149749939', 'name..."
349292708365484032,None,None,Mon Jun 24 22:27:25 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349292708365484032,349292708365484032,K_Mowers,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@K_Mowers I only have done the 90 for 2 days. ...,False,"{'id': 438490210, 'id_str': '438490210', 'name..."
349292441108611072,None,None,Mon Jun 24 22:26:22 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349292441108611072,349292441108611072,No2Speciesism1,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@IntuitiveGal1 I'd seriously suggest coming ba...,False,"{'id': 149749939, 'id_str': '149749939', 'name..."
349292124535132161,None,None,Mon Jun 24 22:25:06 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349292124535132161,349292124535132161,K_Mowers,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@K_Mowers I increased the Cymbalta to 90 mg. M...,False,"{'id': 438490210, 'id_str': '438490210', 'name..."
349291626071457794,None,None,Mon Jun 24 22:23:07 +0000 2013,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,None,349291626071457794,349291626071457794,No2Speciesism1,...,None,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@IntuitiveGal1 Yeah. I'm not doing the crying ...,False,"{'id': 149749939, 'id_str': '149749939', 'name..."
